In [ ]:
#pip install aiohttp
#1 time run

In [ ]:
import csv
import requests
import time
import os
import json
import asyncio
import aiohttp
from aiohttp import ClientSession
import datetime

In [ ]:
tiers=["IRON","BRONZE","SILVER","GOLD","PLATINUM","EMERALD","DIAMOND"]
divisions = ["I", "II", "III", "IV"]
pages=["1","2"]

In [ ]:
def fetch_and_save_ranked_players(tiers, divisions, pages):
    api_key = "RGAPI-9700eac4-1e3c-4da8-92d2-d2168387e08f"  # Use your actual API key
    base_url = "https://euw1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page={page}"
    headers = {
        "X-Riot-Token": api_key
    }
    
    file_path = '../DataProcessing/TestData/rankedPlayers.csv'
    
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['tier', 'division', 'summonerName'])  # CSV Header

        for tier in tiers:
            for division in divisions:
                for page in pages:
                    url = base_url.format(tier=tier, division=division, page=page)
                    response = requests.get(url, headers=headers)

                    # Wait for 2 seconds before making the next request
                    time.sleep(2)

                    if response.status_code == 200:
                        players = response.json()
                        for player in players:
                            if player['wins'] + player['losses'] > 20:
                                writer.writerow([player['tier'], player['rank'], player['summonerName']])
                    else:
                        print(f"Error: {response.status_code}, {response.text}")
# Note: Before running, replace 'Your_API_Key_Here' with your actual API key and ensure the file path for CSV is correct as per your environment.


In [ ]:
#ONLY RUN IF NEW DATA REQUIRED
#made with personal apikey, therefore following personal pullrequest limitation
#fetch_and_save_ranked_players(tiers, divisions, pages)

In [ ]:
#function for finding dublicates

def find_duplicate_summoner_names_with_rows(file_path):
    summoner_occurrences = {}  # Tracks summoner names and their row numbers
    duplicates_info = []  # Stores information about duplicates

    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row_number, row in enumerate(reader, start=1):  # Start counting rows from 1
            summoner_name = row['summonerName']
            if summoner_name in summoner_occurrences:
                # If the summoner name is already encountered, add the current row as a duplicate
                summoner_occurrences[summoner_name].append(row_number)
            else:
                # Otherwise, initialize the list with the current row number
                summoner_occurrences[summoner_name] = [row_number]

    # Filter out summoner names with more than one occurrence and prepare duplicate info
    for summoner_name, row_numbers in summoner_occurrences.items():
        if len(row_numbers) > 1:
            duplicates_info.append((summoner_name, row_numbers))

    return duplicates_info

# Specify the file path
#file_path = '../DataProcessing/TestData/rankedPlayers.csv'
#duplicates = find_duplicate_summoner_names_with_rows(file_path)

# Example output
#for summoner_name, rows in duplicates:
#    print(f"Duplicate: {summoner_name} found in rows: {rows}")


In [ ]:
#function for removing dublicates

def remove_duplicate_summoner_names(file_path):
    processed_summoners = set()  # To track unique summonerNames
    unique_rows = []  # To store rows after removing duplicates

    # Read the file and filter out duplicate summonerNames
    with open(file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            if row['summonerName'] not in processed_summoners:
                processed_summoners.add(row['summonerName'])
                unique_rows.append(row)

    # Write the unique rows back to the file
    with open(file_path, mode='w', newline='', encoding='utf-8') as outfile:
        fieldnames = ['tier', 'division', 'summonerName']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(unique_rows)

# Specify the file path
#file_path = '../DataProcessing/TestData/rankedPlayers.csv'
#remove_duplicate_summoner_names(file_path)


In [ ]:
def split_csv_by_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of CSV writers for each tier
    writers = {}
    files = {}
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            
            tier = row['tier']
            file_name = f"{tier}_players.csv"
            file_path = os.path.join(base_folder, file_name)
            
            # Check if we already have a writer for this tier, if not create it
            if tier not in writers:
                files[tier] = open(file_path, mode='w', newline='', encoding='utf-8')
                writers[tier] = csv.DictWriter(files[tier], fieldnames=reader.fieldnames)
                writers[tier].writeheader()
            
            # Write the row to the appropriate file
            writers[tier].writerow(row)
    
    # Close all the files we've opened
    for file in files.values():
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayers.csv'

# Call the function
split_csv_by_tier(source_file_path)


In [ ]:
# saves summonernames for further processiing
def get_summoner_names_from_csv(file_path):
    summoner_names = []
    
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            summoner_names.append(row['summonerName'])
    
    return summoner_names

# Specify the path to your CSV file
file_path = '../DataProcessing/TestData/DIAMOND_players.csv'
summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)


In [ ]:
#personal apikey function, replacing summonernames with puuid
#FINDS PUUID FROM SUMMONER NAME
def update_summoner_names_with_puuid(file_path, summoner_names):
    api_key = "RGAPI-f1974148-4124-4d03-92aa-b1508bcca232"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    summoner_to_puuid = {}

    start_time = time.time()
    last_update_time = start_time
    total_names = len(summoner_names)
    names_processed = 0

    # Fetch puuid for each summoner name
    for name in summoner_names:
        url = f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}"
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            summoner_to_puuid[name] = data['puuid']
        else:
            print(f"Error fetching data for {name}: {response.status_code}")
        names_processed += 1
        current_time = time.time()

        # Check if 1 minute has passed for progress update
        if current_time - last_update_time >= 60:
            print(f"Progress: {names_processed}/{total_names} summoner names processed.")
            last_update_time = current_time

        time.sleep(1.5)  # Delay to comply with rate limit
    
    # Update progress after finishing all requests
    print(f"Progress: {names_processed}/{total_names} summoner names processed. Update complete.")

    # Now, replace summoner names with puuid in the CSV
    temp_file_path = file_path + ".tmp"
    with open(file_path, mode='r', encoding='utf-8') as infile, open(temp_file_path, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
        writer.writeheader()
        for row in reader:
            if row['summonerName'] in summoner_to_puuid:
                row['summonerName'] = summoner_to_puuid[row['summonerName']]
            writer.writerow(row)
    
    # Replace original file with the updated temp file
    os.replace(temp_file_path, file_path)


# Call the function
update_summoner_names_with_puuid(file_path, summoner_names)



In [ ]:
#FINDS PUUID FROM SUMMONER NAME v2
#with production key
async def fetch_puuid(session, url, headers, semaphore, summoner_to_puuid, name):
    async with semaphore:
        async with session.get(url, headers=headers) as response:
            if response.status == 200:
                data = await response.json()
                summoner_to_puuid[name] = data['puuid']
            else:
                print(f"Error fetching data for {name}: {response.status}")

async def rate_limited_requester(summoner_names, headers, rate_limit=30):
    tasks = []
    summoner_to_puuid = {}

    semaphore = asyncio.Semaphore(rate_limit)

    async with ClientSession() as session:
        for name in summoner_names:
            url = f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}"
            task = asyncio.create_task(fetch_puuid(session, url, headers, semaphore, summoner_to_puuid, name))
            tasks.append(task)

            if len(tasks) % rate_limit == 0:
                await asyncio.sleep(1)

        await asyncio.gather(*tasks)

    return summoner_to_puuid

async def update_summoner_names_with_puuid(file_path, summoner_names):
    api_key = "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }

    summoner_to_puuid = await rate_limited_requester(summoner_names, headers)

    temp_file_path = file_path + ".tmp"
    with open(file_path, mode='r', encoding='utf-8') as infile, open(temp_file_path, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
        writer.writeheader()
        for row in reader:
            if row['summonerName'] in summoner_to_puuid:
                row['summonerName'] = summoner_to_puuid[row['summonerName']]
            writer.writerow(row)
    
    os.replace(temp_file_path, file_path)

# Execution block adapted for Jupyter Notebook
loop = asyncio.get_event_loop()

#source_csv_path = "../DataProcessing/TestData/GOLD_players.csv"


if loop.is_running():
    task = loop.create_task(update_summoner_names_with_puuid(source_csv_path, summoner_names))
else:
    loop.run_until_complete(update_summoner_names_with_puuid(source_csv_path, summoner_names))


In [ ]:
#Finds matches from puuid
# personal api key function
def fetch_matches_and_save(source_csv_path, target_csv_path):
    api_key = "RGAPI-8840c4e8-ef3d-40cb-90fa-2ea005f2bb1c"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    
    start_time = time.time()
    last_update_time = start_time
    summoners_processed = 0

    # Ensure the target file directory exists
    os.makedirs(os.path.dirname(target_csv_path), exist_ok=True)

    with open(source_csv_path, mode='r', encoding='utf-8') as infile, open(target_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        csv_reader = csv.DictReader(infile)
        csv_writer = csv.writer(outfile)
        csv_writer.writerow(['MatchID'])  # Header for target CSV
        
        for row in csv_reader:
            puuid = row['summonerName']  # Assuming this column actually contains puuids
            url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=20"
            response = requests.get(url, headers=headers)
            
            if response.status_code == 200:
                match_ids = response.json()
                for match_id in match_ids:
                    csv_writer.writerow([match_id])
            else:
                print(f"Error fetching matches for {puuid}: {response.status_code}")
            
            summoners_processed += 1
            current_time = time.time()
            
            # Rate limiting
            time.sleep(1.5)
            
            # Print progress every minute
            if current_time - last_update_time >= 60:
                print(f"Progress: {summoners_processed} summoners processed.")
                last_update_time = current_time

    # Final progress update
    print(f"Final Progress: {summoners_processed} summoners processed. Task complete.")

# Example usage
source_csv_path = '../DataProcessing/TestData/DIAMOND_players_korea.csv'
target_csv_path = '../DataProcessing/TestData/DIAMOND_GAMES_KOREA.csv'
fetch_matches_and_save(source_csv_path, target_csv_path)




In [ ]:
#finds dublicate matches
def print_duplicates(csv_file_path):
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Identifying all duplicates, including the first occurrence
    duplicates = df[df.duplicated('MatchID', keep=False)]
    
    # Group by 'MatchID' and aggregate the indexes into a list
    grouped_duplicates = duplicates.groupby('MatchID').apply(lambda x: list(x.index))
    
    # Check if duplicates are found
    if not grouped_duplicates.empty:
        print("Duplicate MatchIDs found along with their locations (indexes):")
        for match_id, indexes in grouped_duplicates.items():
            print(f"MatchID: {match_id}, Indexes: {indexes}")
    else:
        print("No duplicate MatchIDs found.")


In [ ]:
#removing all dublicate matches
import pandas as pd
def remove_duplicates_and_save(csv_file_path, output_file_path):
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Remove duplicates based on the 'MatchID' column and keep the first occurrence
    cleaned_df = df.drop_duplicates('MatchID', keep='first')
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_df.to_csv(output_file_path, index=False)
    
    print(f"Cleaned CSV saved to {output_file_path}.")

In [ ]:
#example usage
#csv_file_path='../DataProcessing/TestData/DIAMOND_GAMES_KOREA.csv'
#output_file_path='../DataProcessing/TestData/CLEANED_DIAMOND_GAMES_KOREA.csv'
#remove_duplicates_and_save(csv_file_path, output_file_path)

In [ ]:
#function to save whole response body riot api timeline
#edit url to desired api url
def call_api_and_save_response_with_progress(csv_file_path, save_folder_path):
    # Define the base URL and headers for the API call
    base_url = "https://asia.api.riotgames.com/lol/match/v5/matches/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"
    }
    
    # Read Match IDs from the CSV file
    with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        match_ids = [row['MatchID'] for row in csv_reader]
    
    total_matches = len(match_ids)
    matches_processed = 0
    start_time = time.time()
    last_update_time = start_time

    # Process each Match ID
    for match_id in match_ids:
        full_url = base_url + match_id + "/timeline"
        
        # Make the API call
        response = requests.get(full_url, headers=headers)
        
        if response.status_code == 200:
            # Save the response body as a JSON file
            file_path = os.path.join(save_folder_path, f"{match_id}.json")
            with open(file_path, 'w', encoding='utf-8') as json_file:
                json.dump(response.json(), json_file, ensure_ascii=False, indent=4)
        else:
            print(f"Failed to fetch data for Match ID {match_id}. Status Code: {response.status_code}")
        
        matches_processed += 1
        
        # Respect the API rate limit
        time.sleep(1/30)  # Sleep to ensure not exceeding 30 calls per second
        
        current_time = time.time()
        if current_time - last_update_time >= 60:
            print(f"Processed {matches_processed}/{total_matches} matches. {int((matches_processed/total_matches)*100)}% complete.")
            last_update_time = current_time
    
    # Final update
    print(f"All {total_matches} matches processed. 100% complete.")

In [ ]:
#function for calling and saving responsebody from postmatch data api
def fetch_and_save_match_data(csv_file_path, output_folder_path):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"
    }

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    # Count total MatchIDs in the CSV file
    with open(csv_file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        total_ids = sum(1 for row in reader)

    # Reset file pointer to start
    with open(csv_file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        current_position = 0
        for row in reader:
            match_id = row[0]
            url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"

            response = requests.get(url, headers=headers)
            current_position += 1
            
            if response.status_code == 200:
                output_file_path = os.path.join(output_folder_path, f"{match_id}.json")
                with open(output_file_path, 'w', encoding='utf-8') as outfile:
                    outfile.write(response.text)
            else:
                print(f"Failed to fetch data for MatchID: {match_id}, Status Code: {response.status_code}")

            # Progress update
            if current_position % 250 == 0 or current_position == total_ids:  # Update every 250 requests or on last request
                print(f"Progress: {current_position}/{total_ids}")

            time.sleep(1 / 30)  # Respect the rate limit

    print(f"Finished processing. Total requests: {current_position}.")